In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os

# U.S. Electricity Generation Technologies and Fuel Consumption Analysis (2015-2023)
## Data Wrangle - Pt. 2

**Collaborators:**

* Amy Zhang (332)
* ChatGPT (0101)
* Perplexity AI (20250328, 20250330)
* Gemini AI (20250328, 20250330, 20250331)

# Table of Contents

## Project Steps

* **Step 1: Aggregated Dataset Creation (agg_table_df) - Re-Processing Original EIA Data (2015-2023)**
* **Step 2: Initial Data Cleaning + Standardization**
    * Remove trailing monthly breakdown columns of Quantities measured in Physical Unit Labels, whilst keeping Physical Unit Label information, and the Year-to-date total. 
* **Step 3: Selecting Relevant Metric Attributes for Analysis**
    * a) Filtering agg_table_df: Complete, Non-Zero Metric Entries
    * b) 'YEAR' completeness check: removing blank enteries
* **Step 4: FINAL SUMMARY OF DATA WRANGLE: filtered_df** 

# Step 1: Aggregated Dataset Creation (agg_table_df) - Re-Processing Original EIA Data (2015-2023)

This step involves re-executing data cleaning procedures on the original merged Power Plant EIA dataset (2015-2023), with a focus on early aggregation. The dataset will be aggregated at the following levels:

* **Reported Prime Mover**
* **Reported Fuel Type**
* **Year**
* **State**

**Key Considerations:**

* **Year-Based Operator Data:** Nine previously removed entries with blank "Operator Name" fields were identified. These entries contain missing year data for existing operators. Although operator names are absent, "Prime Mover" and "Fuel Type" information is retained.
* **Zero-Value Metrics:** All metrics associated with these entries are zero. While seemingly insignificant, these zero values may provide valuable analytical insights.
* **Physical Unit Label Retention:** The "Physical Unit Label" column will be retained, as it provides crucial context for fuel types and material quality trends.

# **Step 2: Initial Data Cleaning and Standardization**
- Removal of monthly quantity breakdown columns (Physical Unit Labels), while retaining the "Physical Unit Label" column and year-to-date totals.

    **Rationale:**
    * The monthly quantity breakdown columns exhibit significant inconsistencies, even after aggregation.
    * The "Quantity" columns should ideally be converted to MMBtu for direct comparison with other energy metrics.
    * If these quantities are initially in other units (e.g., gallons, cubic feet), the conversion to MMBtu may have already been performed.
    * **Conversion Variability:**
        * Conversion factors between fuel quantities and MMBtu vary by fuel type. Common examples include:
            * Natural Gas: 1 cubic foot ≈ 0.00103 MMBtu (approximate, actual value depends on gas composition)
            * Coal: 1 ton ≈ 16-24 MMBtu (varies based on coal type, e.g., bituminous, subbituminous)
            * Oil: 1 gallon ≈ 0.137 MMBtu (for #2 fuel oil)
        * If the "Quantity" columns are already in MMBtu, they can be used directly for comparison. Otherwise, reverse-engineering conversions requires knowledge of fuel types and specific conversion factors.
    * **Importance of "Physical Unit Label":**
        * The "Physical Unit Label" is crucial for normalizing the interpretation of aggregated sums, as different conversion processes are involved for various fuel types.
        * **Example:** Comparing MMBtu values for natural gas and coal requires understanding the original units (cubic feet vs. tons) and the respective conversion processes due to differing energy densities.

    **Key Metrics for Analysis:**
    * **Net Generation (MWh):** Represents electricity supplied to the grid. Negative values may indicate energy storage or grid balancing activities.
    * **Electricity Fuel Consumed (MMBtu):** Energy content of fuels used for electricity generation.
    * **Total Fuel Consumed (MMBtu):** Total energy content of all fuels used (electricity and other purposes).

In [49]:
# Path to your excel file
excel_file_path = '/Users/amyzhang/Desktop/A6_Dashboard/01 Data/exploratory_csv/pivot_table_df.xlsx'

# Import the SECOND sheet
agg_table_df = pd.read_excel(excel_file_path, sheet_name=1)  

# Verify the import
print(agg_table_df.shape)
print(agg_table_df.columns)

(34466, 84)
Index(['YEAR', 'Reported\nFuel Type Code', 'Fuel Type Full Name',
       'Reported\nPrime Mover', 'Prime Mover Full Name', 'Plant State',
       'NERC Region', 'NERC Region Full Name', 'NAICS Code', 'NAICS Full Name',
       'EIA Sector Number', 'Sector Name', 'Physical\nUnit Label',
       'Sum of Net Generation\n(Megawatthours)',
       'Sum of Elec Fuel Consumption\nMMBtu',
       'Sum of Total Fuel Consumption\nMMBtu',
       'Sum of Electric Fuel Consumption\nQuantity',
       'Sum of Total Fuel Consumption\nQuantity', 'Sum of Netgen\nDecember',
       'Sum of Netgen\nNovember', 'Sum of Netgen\nOctober',
       'Sum of Netgen\nSeptember', 'Sum of Netgen\nAugust',
       'Sum of Netgen\nJuly', 'Sum of Netgen\nJune', 'Sum of Netgen\nMay',
       'Sum of Netgen\nApril', 'Sum of Netgen\nMarch',
       'Sum of Netgen\nFebruary', 'Sum of Netgen\nJanuary',
       'Sum of Elec_MMBtu\nDecember', 'Sum of Elec_MMBtu\nNovember',
       'Sum of Elec_MMBtu\nOctober', 'Sum of Elec_MM

In [54]:
# Remove the MMBtuPer_Unit columns
columns_to_remove = [
    'Sum of MMBtuPer_Unit\nJune', 'Sum of MMBtuPer_Unit\nMarch',
    'Sum of MMBtuPer_Unit\nFebruary', 'Sum of MMBtuPer_Unit\nJanuary'
]

agg_table_df = agg_table_df.drop(columns=columns_to_remove, errors='ignore')

# Define the desired column order
desired_column_order = [
    'YEAR', 'Reported\nFuel Type Code', 'Fuel Type Full Name',
    'Reported\nPrime Mover', 'Prime Mover Full Name', 'Plant State','NERC Region',
    'NERC Region Full Name', 'NAICS Code', 'NAICS Full Name',
    'EIA Sector Number', 'Sector Name', 'Physical\nUnit Label',
    'Count of Plant Id',
    'Sum of Conflicting Operator Id Flag',
    'Sum of Net Generation\n(Megawatthours)',
    'Sum of Elec Fuel Consumption\nMMBtu',
    'Sum of Total Fuel Consumption\nMMBtu',
    'Sum of Electric Fuel Consumption\nQuantity',
    'Sum of Total Fuel Consumption\nQuantity',
    'Sum of Quantity\nJanuary',
    'Sum of Quantity\nFebruary',
    'Sum of Quantity\nMarch',
    'Sum of Quantity\nApril',
    'Sum of Quantity\nMay',
    'Sum of Quantity\nJune',
    'Sum of Quantity\nJuly',
    'Sum of Quantity\nAugust',
    'Sum of Quantity\nSeptember',
    'Sum of Quantity\nOctober',
    'Sum of Quantity\nNovember',
    'Sum of Quantity\nDecember',
    'Sum of Elec_Quantity\nJanuary',
    'Sum of Elec_Quantity\nFebruary',
    'Sum of Elec_Quantity\nMarch',
    'Sum of Elec_Quantity\nApril',
    'Sum of Elec_Quantity\nMay',
    'Sum of Elec_Quantity\nJune',
    'Sum of Elec_Quantity\nJuly',
    'Sum of Elec_Quantity\nAugust',
    'Sum of Elec_Quantity\nSeptember',
    'Sum of Elec_Quantity\nOctober',
    'Sum of Elec_Quantity\nNovember',
    'Sum of Elec_Quantity\nDecember',
    'Sum of Tot_MMBtu\nJanuary',
    'Sum of Tot_MMBtu\nFebruary',
    'Sum of Tot_MMBtu\nMarch',
    'Sum of Tot_MMBtu\nApril',
    'Sum of Tot_MMBtu\nMay',
    'Sum of Tot_MMBtu\nJune',
    'Sum of Tot_MMBtu\nJuly',
    'Sum of Tot_MMBtu\nAugust',
    'Sum of Tot_MMBtu\nSeptember',
    'Sum of Tot_MMBtu\nOctober',
    'Sum of Tot_MMBtu\nNovember',
    'Sum of Tot_MMBtu\nDecember',
    'Sum of Elec_MMBtu\nJanuary',
    'Sum of Elec_MMBtu\nFebruary',
    'Sum of Elec_MMBtu\nMarch',
    'Sum of Elec_MMBtu\nApril',
    'Sum of Elec_MMBtu\nMay',
    'Sum of Elec_MMBtu\nJune',
    'Sum of Elec_MMBtu\nJuly',
    'Sum of Elec_MMBtu\nAugust',
    'Sum of Elec_MMBtu\nSeptember',
    'Sum of Elec_MMBtu\nOctober',
    'Sum of Elec_MMBtu\nNovember',
    'Sum of Elec_MMBtu\nDecember',
    'Sum of Netgen\nJanuary',
    'Sum of Netgen\nFebruary',
    'Sum of Netgen\nMarch',
    'Sum of Netgen\nApril',
    'Sum of Netgen\nMay',
    'Sum of Netgen\nJune',
    'Sum of Netgen\nJuly',
    'Sum of Netgen\nAugust',
    'Sum of Netgen\nSeptember',
    'Sum of Netgen\nOctober',
    'Sum of Netgen\nNovember',
    'Sum of Netgen\nDecember'
]

agg_table_df = agg_table_df[desired_column_order]

## Imputing 'Physical Unit Label'

In [56]:
imputation_map = {
    'OBG': 'Mcf',
    'DFO': 'barrels',
    'RFO': 'barrels',
    'WO': 'barrels',
    'JF': 'barrels',
    'KER': 'barrels',
    'OBL': 'barrels',
    'WDL': 'barrels',
    'NG': 'mcf',
    'PG': 'mcf',
    'LFG': 'mcf',
    'SGC': 'mcf',
    'OG': 'mcf',
    'BFG': 'mcf',
    'SGP': 'mcf',
    'WAT': 'megawatthours',
    'MWH': 'megawatthours',
    'BIT': 'short tons',
    'WDS': 'short tons',
    'SUB': 'short tons',
    'LIG': 'short tons',
    'AB': 'short tons',
    'PC': 'short tons',
    'ANT': 'short tons',
    'OBS': 'short tons',
    'TDF': 'short tons',
    'MSB': 'short tons',
    'MSN': 'short tons',
    'SC': 'short tons',
    'WC': 'short tons',
    'BLQ': 'short tons',
    'SLW': 'short tons',
    'RC': 'short tons'
}

imputation_map_mwh = {
    'SUN': 'megawatthours',
    'GEO': 'megawatthours',
    'NUC': 'megawatthours',
    'OTH': 'megawatthours',
    'PUR': 'megawatthours',
    'WH': 'megawatthours',
    'WND': 'megawatthours',
    'H2': 'megawatthours'
}

def impute_physical_unit_strict(row):
    if row['Reported\nFuel Type Code'] in imputation_map:
        return imputation_map[row['Reported\nFuel Type Code']]
    elif row['Reported\nFuel Type Code'] in imputation_map_mwh:
        return imputation_map_mwh[row['Reported\nFuel Type Code']]
    else:
        return np.nan # Set to NaN if not in the map

# Apply the imputation function
agg_table_df['Physical\nUnit Label'] = agg_table_df.apply(impute_physical_unit_strict, axis=1)

# Print the updated Physical\nUnit Label counts to verify
print(agg_table_df['Physical\nUnit Label'].value_counts(dropna=False))

Physical\nUnit Label
mcf              12361
barrels           8623
short tons        6696
megawatthours     6096
Mcf                690
Name: count, dtype: int64


# Step 3: Selecting Relevant Metric Attributes for Analysis

In [58]:
# Count of missing values per column
missing_count = agg_table_df.isnull().sum()

# Percentage of missing values per column
missing_percentage = (missing_count / len(agg_table_df)) * 100

# Combine into a single DataFrame for better readability
missing_summary = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage': missing_percentage
})

# Display the summary
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(missing_summary.to_string())

                                            Missing Count  Missing Percentage
YEAR                                                    0            0.000000
Reported\nFuel Type Code                                0            0.000000
Fuel Type Full Name                                     0            0.000000
Reported\nPrime Mover                                   0            0.000000
Prime Mover Full Name                                   0            0.000000
Plant State                                             0            0.000000
NERC Region                                             0            0.000000
NERC Region Full Name                                1646            4.775721
NAICS Code                                              0            0.000000
NAICS Full Name                                         0            0.000000
EIA Sector Number                                       0            0.000000
Sector Name                                             0       

In [60]:
agg_table_df.describe()

,NAICS Code,EIA Sector Number,Count of Plant Id,Sum of Conflicting Operator Id Flag,Sum of Net Generation\n(Megawatthours),Sum of Elec Fuel Consumption\nMMBtu,Sum of Total Fuel Consumption\nMMBtu,Sum of Electric Fuel Consumption\nQuantity,Sum of Total Fuel Consumption\nQuantity,Sum of Quantity\nJanuary,...,Sum of Netgen\nMarch,Sum of Netgen\nApril,Sum of Netgen\nMay,Sum of Netgen\nJune,Sum of Netgen\nJuly,Sum of Netgen\nAugust,Sum of Netgen\nSeptember,Sum of Netgen\nOctober,Sum of Netgen\nNovember,Sum of Netgen\nDecember
count,34466.000000,34466.000000,34466.000000,34466.000000,3.446600e+04,3.446600e+04,3.446600e+04,3.446600e+04,3.446600e+04,3.281000e+04,...,3.276800e+04,3.277800e+04,3.278600e+04,3.281200e+04,3.280700e+04,3.280600e+04,3.282800e+04,3.285700e+04,3.290800e+04,3.312200e+04
mean,37150.028782,3.642198,3.819184,0.048454,1.074573e+06,9.603531e+06,1.031287e+07,3.294411e+06,3.840968e+06,3.215886e+05,...,8.771390e+04,8.113740e+04,8.927486e+04,9.991499e+04,1.128640e+05,1.110525e+05,9.590137e+04,8.716095e+04,8.516102e+04,9.337230e+04
std,105091.786835,2.356452,16.576913,0.493424,5.121576e+06,4.963607e+07,4.989896e+07,2.548408e+07,2.684487e+07,2.203266e+06,...,4.243889e+05,3.983298e+05,4.422754e+05,4.715706e+05,5.085928e+05,5.017500e+05,4.426034e+05,4.115501e+05,4.031862e+05,4.440034e+05
min,22.000000,1.000000,1.000000,0.000000,-1.247579e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-1.572970e+05,-8.172400e+04,-1.163920e+05,-1.510030e+05,-1.567240e+05,-1.627240e+05,-1.474190e+05,-1.376330e+05,-1.354380e+05,-1.620860e+05
25%,22.000000,1.000000,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,22.000000,3.000000,1.000000,0.000000,8.670192e+03,6.409100e+04,1.167815e+05,2.104000e+03,4.972500e+03,3.250000e+02,...,6.400000e+02,5.620260e+02,6.280900e+02,7.020465e+02,7.903720e+02,7.555655e+02,6.561425e+02,6.130000e+02,5.921835e+02,6.661260e+02
75%,622.000000,6.000000,2.000000,0.000000,1.432190e+05,1.004034e+06,2.124156e+06,2.435920e+05,5.273208e+05,4.688025e+04,...,1.172954e+04,1.087221e+04,1.166849e+04,1.297609e+04,1.479074e+04,1.441782e+04,1.246893e+04,1.160484e+04,1.128661e+04,1.214742e+04
max,562213.000000,7.000000,711.000000,26.000000,1.156403e+08,1.216461e+09,1.216461e+09,1.188970e+09,1.188970e+09,8.968791e+07,...,1.098051e+07,1.147359e+07,1.200400e+07,1.070404e+07,1.196312e+07,1.224520e+07,1.073942e+07,9.574459e+06,8.853693e+06,9.329821e+06


In [63]:
# Export data to CSV
path = '/Users/amyzhang/Desktop/A6_Dashboard/01 Data/exploratory_csv'
output_file = 'agg_table_df.csv'
full_path = os.path.join(path, output_file)

# Save the dataframe
agg_table_df.to_csv(full_path, index=False)

# Check if the file was created successfully
if os.path.exists(full_path):
    print(f"Saving confirmed: '{output_file}' has been created successfully.")
else:
    print("Error: File was not saved.")

Saving confirmed: 'agg_table_df.csv' has been created successfully.


In [67]:
columns_to_remove = [
    'Sum of Quantity\nJanuary',
    'Sum of Quantity\nFebruary', 'Sum of Quantity\nMarch',
    'Sum of Quantity\nApril', 'Sum of Quantity\nMay',
    'Sum of Quantity\nJune', 'Sum of Quantity\nJuly',
    'Sum of Quantity\nAugust', 'Sum of Quantity\nSeptember',
    'Sum of Quantity\nOctober', 'Sum of Quantity\nNovember',
    'Sum of Quantity\nDecember', 'Sum of Elec_Quantity\nJanuary',
    'Sum of Elec_Quantity\nFebruary', 'Sum of Elec_Quantity\nMarch',
    'Sum of Elec_Quantity\nApril', 'Sum of Elec_Quantity\nMay',
    'Sum of Elec_Quantity\nJune', 'Sum of Elec_Quantity\nJuly',
    'Sum of Elec_Quantity\nAugust', 'Sum of Elec_Quantity\nSeptember',
    'Sum of Elec_Quantity\nOctober', 'Sum of Elec_Quantity\nNovember',
    'Sum of Elec_Quantity\nDecember'
]

agg_table_df = agg_table_df.drop(columns=columns_to_remove, errors='ignore')

# Verify the removal
print(agg_table_df.columns) #Print the remaining columns.

Index(['YEAR', 'Reported\nFuel Type Code', 'Fuel Type Full Name',
       'Reported\nPrime Mover', 'Prime Mover Full Name', 'Plant State',
       'NERC Region', 'NERC Region Full Name', 'NAICS Code', 'NAICS Full Name',
       'EIA Sector Number', 'Sector Name', 'Physical\nUnit Label',
       'Count of Plant Id', 'Sum of Conflicting Operator Id Flag',
       'Sum of Net Generation\n(Megawatthours)',
       'Sum of Elec Fuel Consumption\nMMBtu',
       'Sum of Total Fuel Consumption\nMMBtu',
       'Sum of Electric Fuel Consumption\nQuantity',
       'Sum of Total Fuel Consumption\nQuantity', 'Sum of Tot_MMBtu\nJanuary',
       'Sum of Tot_MMBtu\nFebruary', 'Sum of Tot_MMBtu\nMarch',
       'Sum of Tot_MMBtu\nApril', 'Sum of Tot_MMBtu\nMay',
       'Sum of Tot_MMBtu\nJune', 'Sum of Tot_MMBtu\nJuly',
       'Sum of Tot_MMBtu\nAugust', 'Sum of Tot_MMBtu\nSeptember',
       'Sum of Tot_MMBtu\nOctober', 'Sum of Tot_MMBtu\nNovember',
       'Sum of Tot_MMBtu\nDecember', 'Sum of Elec_MMBtu\nJan

## 3a) Filtering agg_table_df: Complete, Non-Zero Metric Entries

* **Objective:** Filter `agg_table_df` to include only rows with complete metric entries (no missing values) and at least one non-zero metric.
* **Verification:** Confirm that no Prime Mover technologies are excluded by this filter.
* **Action:** Apply the filter to `agg_table_df`.

In [71]:
# List of metric columns
metric_columns = [
    'Sum of Net Generation\n(Megawatthours)',
       'Sum of Elec Fuel Consumption\nMMBtu',
       'Sum of Total Fuel Consumption\nMMBtu',
       'Sum of Electric Fuel Consumption\nQuantity',
       'Sum of Total Fuel Consumption\nQuantity', 'Sum of Tot_MMBtu\nJanuary',
       'Sum of Tot_MMBtu\nFebruary', 'Sum of Tot_MMBtu\nMarch',
       'Sum of Tot_MMBtu\nApril', 'Sum of Tot_MMBtu\nMay',
       'Sum of Tot_MMBtu\nJune', 'Sum of Tot_MMBtu\nJuly',
       'Sum of Tot_MMBtu\nAugust', 'Sum of Tot_MMBtu\nSeptember',
       'Sum of Tot_MMBtu\nOctober', 'Sum of Tot_MMBtu\nNovember',
       'Sum of Tot_MMBtu\nDecember', 'Sum of Elec_MMBtu\nJanuary',
       'Sum of Elec_MMBtu\nFebruary', 'Sum of Elec_MMBtu\nMarch',
       'Sum of Elec_MMBtu\nApril', 'Sum of Elec_MMBtu\nMay',
       'Sum of Elec_MMBtu\nJune', 'Sum of Elec_MMBtu\nJuly',
       'Sum of Elec_MMBtu\nAugust', 'Sum of Elec_MMBtu\nSeptember',
       'Sum of Elec_MMBtu\nOctober', 'Sum of Elec_MMBtu\nNovember',
       'Sum of Elec_MMBtu\nDecember', 'Sum of Netgen\nJanuary',
       'Sum of Netgen\nFebruary', 'Sum of Netgen\nMarch',
       'Sum of Netgen\nApril', 'Sum of Netgen\nMay', 'Sum of Netgen\nJune',
       'Sum of Netgen\nJuly', 'Sum of Netgen\nAugust',
       'Sum of Netgen\nSeptember', 'Sum of Netgen\nOctober',
       'Sum of Netgen\nNovember', 'Sum of Netgen\nDecember'
]

# Create a boolean mask for rows with no missing values in metric columns
complete_metrics_mask = agg_table_df[metric_columns].notna().all(axis=1)

# Create a boolean mask for rows where at least one metric is not zero
non_zero_metrics_mask = (agg_table_df[metric_columns] != 0).any(axis=1)

# Combine the masks
final_mask = complete_metrics_mask & non_zero_metrics_mask

# Create the filtered DataFrame
filtered_df = agg_table_df[final_mask].copy() #Use copy to avoid SettingWithCopyWarning.

# Print the size of the filtered DataFrame
print(f"Shape of filtered DataFrame: {filtered_df.shape}")

Shape of filtered DataFrame: (25574, 56)


In [73]:
agg_table_df.shape

(34466, 56)

In [75]:
# Count of missing values per column
missing_count = filtered_df.isnull().sum()

# Percentage of missing values per column
missing_percentage = (missing_count / len(filtered_df)) * 100

# Combine into a single DataFrame for better readability
missing_summary = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage': missing_percentage
})

# Display the summary
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(missing_summary.to_string())

                                            Missing Count  Missing Percentage
YEAR                                                    0            0.000000
Reported\nFuel Type Code                                0            0.000000
Fuel Type Full Name                                     0            0.000000
Reported\nPrime Mover                                   0            0.000000
Prime Mover Full Name                                   0            0.000000
Plant State                                             0            0.000000
NERC Region                                             0            0.000000
NERC Region Full Name                                 302            1.180887
NAICS Code                                              0            0.000000
NAICS Full Name                                         0            0.000000
EIA Sector Number                                       0            0.000000
Sector Name                                             0       

In [79]:
filtered_df['Prime Mover Full Name'].value_counts(dropna=False)

Prime Mover Full Name
Steam Turbine                                   9553
Internal Combustion Engine                      3722
Combustion (Gas) Turbine                        3609
Combined-Cycle -- Steam Part                    1860
Combined-Cycle Combustion Turbine Part          1833
Photovoltaic                                    1803
Hydraulic Turbine                               1101
Wind Turbine, Onshore                            994
Energy Storage, Battery                          354
Fuel Cell                                        213
Energy Storage, Reversible Hydraulic Turbine     179
Combined-Cycle Single-Shaft                      137
Other                                             89
Turbines Used in a Binary Cycle (Geothermal)      65
Energy Storage, Flywheel                          27
Energy Storage, Concentrated Solar Power          16
Wind Turbine, Offshore                            10
Energy Storage, Compressed Air                     9
Name: count, dtype: int6

In [84]:
agg_table_df['Prime Mover Full Name'].value_counts(dropna=False)

Prime Mover Full Name
Steam Turbine                                   13608
Combustion (Gas) Turbine                         5001
Internal Combustion Engine                       4935
Combined-Cycle Combustion Turbine Part           2686
Combined-Cycle -- Steam Part                     2428
Photovoltaic                                     2039
Hydraulic Turbine                                1156
Wind Turbine, Onshore                            1074
Energy Storage, Battery                           557
Fuel Cell                                         299
Energy Storage, Reversible Hydraulic Turbine      191
Other                                             182
Combined-Cycle Single-Shaft                       170
Turbines Used in a Binary Cycle (Geothermal)       67
Energy Storage, Flywheel                           34
Energy Storage, Concentrated Solar Power           18
Wind Turbine, Offshore                             11
Energy Storage, Compressed Air                      9
(blank

**Overall Metrics:**

* **`Sum of Net Generation\n(Megawatthours)`**:
    * This represents the total amount of electricity generated by a power plant, measured in megawatt-hours (MWh). It's the net output, meaning it accounts for electricity used by the plant itself.
    * Essentially, it's the total electricity sold to the grid.
* **`Sum of Elec Fuel Consumption\nMMBtu`**:
    * This is the total amount of fuel consumed by the power plant specifically for electricity generation, measured in million British thermal units (MMBtu).
    * MMBtu is a measure of the heat content of the fuel.
* **`Sum of Total Fuel Consumption\nMMBtu`**:
    * This is the total amount of fuel consumed by the power plant for all purposes, including electricity generation and any other on-site uses, also measured in MMBtu.
* **`Sum of Electric Fuel Consumption\nQuantity`**:
    * This is the total quantity of fuel (e.g., in tons, barrels, cubic feet) consumed by the power plant for electricity generation. The unit of measurement depends on the fuel type.
* **`Sum of Total Fuel Consumption\nQuantity`**:
    * This is the total quantity of fuel consumed by the power plant for all purposes. The unit of measurement depends on the fuel type.

**Monthly Metrics:**

* **`Sum of Tot_MMBtu\n[Month]`**:
    * The total MMBtu of fuel consumed during the specified month.
* **`Sum of Elec_MMBtu\n[Month]`**:
    * The MMBtu of fuel consumed specifically for electricity generation during the specified month.
* **`Sum of Netgen\n[Month]`**:
    * The net electricity generation (in MWh) during the specified month.

**General Notes:**

* **MMBtu (Million British Thermal Units):** This is a standard unit for measuring the heat content of fuels.
* **Megawatthours (MWh):** This is the standard unit for measuring electricity generation.
* **Quantity:** The units for the quantity metrics will vary based on the fuel type. For example, coal is often measured in tons, natural gas in cubic feet, and oil in barrels.
* The Sum prefix means that the values have been aggregated to a higher level of grouping.

In [89]:
metric_columns = [
    'Sum of Net Generation\n(Megawatthours)',
    'Sum of Elec Fuel Consumption\nMMBtu',
    'Sum of Total Fuel Consumption\nMMBtu',
    'Sum of Electric Fuel Consumption\nQuantity',
    'Sum of Total Fuel Consumption\nQuantity',
    'Sum of Tot_MMBtu\nJanuary',
    'Sum of Tot_MMBtu\nFebruary', 'Sum of Tot_MMBtu\nMarch',
    'Sum of Tot_MMBtu\nApril', 'Sum of Tot_MMBtu\nMay',
    'Sum of Tot_MMBtu\nJune', 'Sum of Tot_MMBtu\nJuly',
    'Sum of Tot_MMBtu\nAugust', 'Sum of Tot_MMBtu\nSeptember',
    'Sum of Tot_MMBtu\nOctober', 'Sum of Tot_MMBtu\nNovember',
    'Sum of Tot_MMBtu\nDecember', 'Sum of Elec_MMBtu\nJanuary',
    'Sum of Elec_MMBtu\nFebruary', 'Sum of Elec_MMBtu\nMarch',
    'Sum of Elec_MMBtu\nApril', 'Sum of Elec_MMBtu\nMay',
    'Sum of Elec_MMBtu\nJune', 'Sum of Elec_MMBtu\nJuly',
    'Sum of Elec_MMBtu\nAugust', 'Sum of Elec_MMBtu\nSeptember',
    'Sum of Elec_MMBtu\nOctober', 'Sum of Elec_MMBtu\nNovember',
    'Sum of Elec_MMBtu\nDecember', 'Sum of Netgen\nJanuary',
    'Sum of Netgen\nFebruary', 'Sum of Netgen\nMarch',
    'Sum of Netgen\nApril', 'Sum of Netgen\nMay', 'Sum of Netgen\nJune',
    'Sum of Netgen\nJuly', 'Sum of Netgen\nAugust',
    'Sum of Netgen\nSeptember', 'Sum of Netgen\nOctober',
    'Sum of Netgen\nNovember', 'Sum of Netgen\nDecember'
]

# Get the describe() output for the specified columns
description = filtered_df[metric_columns].describe()

# Print the description
print(description)

       Sum of Net Generation\n(Megawatthours)  \
count                            2.557400e+04   
mean                             1.443153e+06   
std                              5.900136e+06   
min                             -1.247579e+06   
25%                              2.891805e+03   
50%                              3.683124e+04   
75%                              2.992633e+05   
max                              1.156403e+08   

       Sum of Elec Fuel Consumption\nMMBtu  \
count                         2.557400e+04   
mean                          1.289973e+07   
std                           5.724670e+07   
min                           0.000000e+00   
25%                           2.025250e+04   
50%                           2.618760e+05   
75%                           2.122903e+06   
max                           1.216461e+09   

       Sum of Total Fuel Consumption\nMMBtu  \
count                          2.557400e+04   
mean                           1.384572e+07   
st

## 3b) Check for YEAR completness; remove 77 rows

In [95]:
filtered_df['YEAR'].value_counts(dropna=False)

YEAR
2018       2878
2019       2861
2021       2846
2022       2844
2016       2843
2023       2833
2017       2814
2020       2811
2015       2767
(blank)      77
Name: count, dtype: int64

In [97]:
# Store value counts before removal
prime_mover_before = filtered_df['Prime Mover Full Name'].value_counts(dropna=False)
reported_fuel_before = filtered_df['Fuel Type Full Name'].value_counts(dropna=False)

# Remove rows where 'YEAR' is '(blank)'
filtered_df = filtered_df[filtered_df['YEAR'] != '(blank)']

# Store value counts after removal
prime_mover_after = filtered_df['Prime Mover Full Name'].value_counts(dropna=False)
reported_fuel_after = filtered_df['Fuel Type Full Name'].value_counts(dropna=False)

# Compare value counts
print("Prime Mover Full Name - Before Removal:")
print(prime_mover_before)
print("\nPrime Mover Full Name - After Removal:")
print(prime_mover_after)

print("\nFuel Tpe Full Name - Before Removal:")
print(reported_fuel_before)
print("\nFuel Type Full Name - After Removal:")
print(reported_fuel_after)

Prime Mover Full Name - Before Removal:
Prime Mover Full Name
Steam Turbine                                   9553
Internal Combustion Engine                      3722
Combustion (Gas) Turbine                        3609
Combined-Cycle -- Steam Part                    1860
Combined-Cycle Combustion Turbine Part          1833
Photovoltaic                                    1803
Hydraulic Turbine                               1101
Wind Turbine, Onshore                            994
Energy Storage, Battery                          354
Fuel Cell                                        213
Energy Storage, Reversible Hydraulic Turbine     179
Combined-Cycle Single-Shaft                      137
Other                                             89
Turbines Used in a Binary Cycle (Geothermal)      65
Energy Storage, Flywheel                          27
Energy Storage, Concentrated Solar Power          16
Wind Turbine, Offshore                            10
Energy Storage, Compressed Air       

**Impact of Removing Rows with 'YEAR' as '(blank)'**

**Prime Mover Full Name:**

* The removal has a **minimal impact** on the distribution of 'Prime Mover Full Name' categories.
* The counts for each category **decrease slightly**, reflecting the removal of the 77 rows, but the overall proportions remain very similar.
* This suggests that the '(blank)' year rows were **relatively evenly distributed** across the different prime mover types.

**Reported Fuel Full Name:**

* Similar to the prime mover, the fuel type categories are **only slightly altered**.
* The counts for most fuel types **decrease slightly**, as expected.
* The `(blank)` fuel type category is unchanged, because removing the blank year, does not remove the blank fuel type.
* The overall distribution of fuel types remains **largely consistent**.
* This indicates that the '(blank)' year rows did not significantly bias the representation of any particular fuel type.

**Overall Impact:**

* The removal of the '(blank)' year rows has a **negligible effect** on the overall distribution of 'Prime Mover Full Name' and 'Reported Fuel Full Name' categories.
* This suggests that the '(blank)' year rows were **not concentrated** in any specific category, and their removal does not introduce any significant bias.
* Therefore, it seems **safe to remove** the rows where year is blank.

In [100]:
# Remove rows where 'YEAR' is '(blank)'
filtered_df = filtered_df[filtered_df['YEAR'] != '(blank)']

# Verify the removal
print(filtered_df['YEAR'].value_counts(dropna=False)) #Verify the blank rows have been removed.

YEAR
2018    2878
2019    2861
2021    2846
2022    2844
2016    2843
2023    2833
2017    2814
2020    2811
2015    2767
Name: count, dtype: int64


# Step 4) Summary of Data Wrangling - Final Steps in Wrangling the EIA Dataset: Focus on Filtered Data

In the final steps of wrangling the EIA dataset, we revisited the original merge of EIA datasets (2015-2023) and created a pivot table in Excel, aggregated at the Year-State-Prime Mover-Fuel Type level. This aggregation helped streamline our focus by emphasizing **MMBtu conversions** of the **Quantity** columns. The **Quantity** values can be reverse-engineered using the **Physical Unit Labels**, and we've cross-checked them with the **Year-To-Date Quantity Metric** (which we've retained in the dataset).

Our primary goal is to understand the **technology** behind the energy data rather than the specific operators or individual plants. To achieve this, we decided to filter the dataset to only include rows with **complete metric data** (i.e., no missing values) and where **entries are not zero across the board**. This ensures that the dataset we work with provides the most **reliable and meaningful insights** on energy generation and consumption metrics.

As a result, we've removed approximately **29%** of the rows (from 34,446 to 25,574 rows), but importantly, **we retain all the original Prime Movers** from the aggregated table. This approach allows us to focus on high-quality, complete data, while maintaining the necessary granularity to analyze the underlying technology.

This filtered dataframe is the most accurate representation of the energy data for our analysis moving forward.


In [102]:
# Export data to CSV
path = '/Users/amyzhang/Desktop/A6_Dashboard/01 Data/exploratory_csv'
output_file = 'filtered_agg_df.csv'
full_path = os.path.join(path, output_file)

# Save the dataframe
filtered_df.to_csv(full_path, index=False)

# Check if the file was created successfully
if os.path.exists(full_path):
    print(f"Saving confirmed: '{output_file}' has been created successfully.")
else:
    print("Error: File was not saved.")

Saving confirmed: 'filtered_agg_df.csv' has been created successfully.
